In [ ]:
import os 
import numpy as np
from pcraster import *
from pcraster.framework import *
import onnxruntime as ort
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
os.chdir(r"/Users/cr/Documents/UU/Master/thesis/life")

# Perform game of life with pcraster

In [ ]:
def pcr_GameOfLife(self, output_map = str): 
    aliveScalar = scalar(self.alive)
    numberOfAliveNeighbours = windowtotal(aliveScalar, 3) - aliveScalar

    threeAliveNeighbours = numberOfAliveNeighbours == 3
    birth = pcrand(threeAliveNeighbours, pcrnot(self.alive))

    survivalA = pcrand((numberOfAliveNeighbours == 2), self.alive)
    survivalB = pcrand((numberOfAliveNeighbours == 3), self.alive)
    survival = pcror(survivalA, survivalB)
    self.alive = pcror(birth, survival)

    self.alive = ifthen(self.defined, self.alive)
    self.report(self.alive, output_map)

In [ ]:
seed_number = 1234
alive_rate = 0.15
plot_time_step = 10

class GameOfLife(DynamicModel):
    def __init__(self):
        DynamicModel.__init__(self)
        setclone('clone.map')
 
    def initial(self):
        setclone(20, 20, 1.0, 0, 0)
        
        pcraster._pcraster.setrandomseed(seed_number) # set seed
        aUniformMap = uniform(1)
        self.report(aUniformMap,'aUniformMap')
        self.alive = aUniformMap < alive_rate

        # report the alive cells
        self.report(self.alive, 'testpcr')
        self.alive = self.alive == 1
        self.defined = defined(self.alive)
 
    def dynamic(self):
        pcr_GameOfLife(self, 'testpcr')
 
nrOfTimeSteps = 100
myModel = GameOfLife()
dynamicModel = DynamicFramework(myModel, nrOfTimeSteps)
dynamicModel.run()

alive = readmap('testpcr.map')
aliveml_np = pcr2numpy(alive, 2)

plt.figure()
plt.imshow(aliveml_np)
plt.title(f'PCRaster - Time Step 0')
plt.show()

# visualize the results for each time step
for i in range(plot_time_step):
    # read the PCRaster map file for the current time step
    raster = readmap(f'testpcr0.{i+1:03d}')

    # retrieve the raster data as a numpy array
    raster_data = pcr2numpy(raster, 2)

    # plot the raster data for the current time step
    plt.figure()
    plt.imshow(raster_data)
    plt.title(f'PCRaster - Time Step {i+1}')
    plt.show()

# Perform game of life with ML model

In [ ]:
def runtime_GameOfLife(self, output_map = str):  
    self.input_data = pcr2numpy(self.alive, 0).astype(np.float32).reshape(1, 20, 20, 1)
    ort_inputs = {self.input_name: self.input_data}
    ort_outs = self.ort_session.run(None, ort_inputs)
    self.alive_np = (ort_outs[0] > 0.5).astype(np.float32).reshape(20, 20)
    self.alive = numpy2pcr(Scalar, self.alive_np, 241)
    self.alive = ifthen(self.defined, self.alive)
    self.report(self.alive, output_map)

### Data size 200

In [ ]:
class GameOfLife(DynamicModel):
  def __init__(self):
    DynamicModel.__init__(self)
    setclone('clone.map')
  
  def initial(self):

    setclone(20, 20, 1.0, 0, 0)
    
    pcraster._pcraster.setrandomseed(seed_number) # set seed
    aUniformMap = uniform(1)
    self.report(aUniformMap,'aUniformMap')
    self.alive = aUniformMap < alive_rate

    # report the alive cells
    self.report(self.alive, 'th')
    self.alive = self.alive == 1
    self.defined = defined(self.alive)    

    self.ort_session = ort.InferenceSession("cnn_game_of_life200.onnx")
    self.input_name = self.ort_session.get_inputs()[0].name

  def dynamic(self):
    runtime_GameOfLife(self, 'th')

nrOfTimeSteps = 100
myModel = GameOfLife()
dynamicModel = DynamicFramework(myModel, nrOfTimeSteps)
dynamicModel.run()

alive = readmap('th.map')
aliveml_np = pcr2numpy(alive, 2)

plt.figure()
plt.imshow(aliveml_np)
plt.title(f'ML model 200 - Time Step 0')
plt.show()

# visualize the results for each time step
for i in range(plot_time_step):
    # read the PCRaster map file for the current time step
    raster = readmap(f'th000000.{i+1:03d}')

    # retrieve the raster data as a numpy array
    raster_data = pcr2numpy(raster, 2)

    # plot the raster data for the current time step
    plt.figure()
    plt.imshow(raster_data)
    plt.title(f'ML model 200 - Time Step {i+1}')
    plt.show()

In [ ]:
y_true = np.zeros(shape=(100, 20, 20))
y_pred = np.zeros(shape=(100, 20, 20))

for i in range(1, 101): 
    # True labels
    true = readmap(f'testpcr0.{i:03d}')
    y_true[i-1] = pcr2numpy(true, 999)
    # Predicted labels
    pred = readmap(f'th000000.{i:03d}')
    y_pred[i-1] = pcr2numpy(pred, 999)


y_true = y_true.reshape(-1)
y_pred = y_pred.reshape(-1)

# Calculate confusion matrix
cm = confusion_matrix(y_true, y_pred)
tn, fp, fn, tp = cm.ravel()

# Calculate metrics
accuracy = (tp + tn) / (tp + tn + fn + fp)
recall = tp / (tp + fn)
specificity = tn / (tn + fp)

# Print the results
print(f"Accuracy: {np.mean(accuracy):.2f}")
print(f"Recall: {np.mean(recall):.2f}")
print(f"Specificity: {np.mean(specificity):.2f}")

In [ ]:
accuracy_sum200 = []
specificity_sum200 = []
recall_sum200 = []

for i in range(1, 101):
    print('time step:', i)
    truth = readmap(f'testpcr0.{i:03d}')
    truth_np = pcr2numpy(truth, 2)

    test = readmap(f'th000000.{i:03d}')
    test_np = pcr2numpy(test, 2)

    # confusion matrix
    tn, fp, fn, tp = confusion_matrix(truth_np.flatten(), test_np.flatten()).ravel()

    # accuracy
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    accuracy_sum200.append(accuracy)

    # recall
    recall = tp / (tp + fn)
    recall_sum200.append(recall)

    # specificity
    specificity = tn / (tn + fp)
    specificity_sum200.append(specificity)

    print(f"Accuracy: {round(accuracy, 2)}")
    print(f"Recall: {round(recall, 2)}")
    print(f"Specificity: {round(specificity, 2)}")
    print(' ')

print("Average accuracy:", round(np.mean(accuracy_sum200), 2))
print("Average recall:", round(np.mean(recall_sum200), 2))
print("Average Specificity:", round(np.mean(specificity_sum200), 2))

In [ ]:
# Plotting the accuracy data
plt.figure(figsize=(10, 5))
plt.plot(accuracy_sum200, marker='o', linestyle='-', color='b')
plt.title('Model Accuracy Over Time - Data Size 200')
plt.xlabel('Time Step')
plt.ylabel('Accuracy')
plt.grid(True)

# Plotting the recall data
plt.figure(figsize=(10, 5))
plt.plot(recall_sum200, marker='o', linestyle='-', color='r')
plt.title('Model Recall Over Time - Data Size 200')
plt.xlabel('Time Step')
plt.ylabel('Recall')
plt.grid(True)

# Plotting the specificity data
plt.figure(figsize=(10, 5))
plt.plot(specificity_sum200, marker='o', linestyle='-', color='g')
plt.title('Model Specificity Over Time - Data Size 200')
plt.xlabel('Time step')
plt.ylabel('Specificity')
plt.grid(True)
plt.show()

### Data size 2000

In [ ]:
class GameOfLife(DynamicModel):
  def __init__(self):
    DynamicModel.__init__(self)
    setclone('clone.map')
  
  def initial(self):
    setclone(20, 20, 1.0, 0, 0)
    
    pcraster._pcraster.setrandomseed(seed_number) # set seed
    aUniformMap = uniform(1)
    self.report(aUniformMap,'aUniformMap')
    self.alive = aUniformMap < alive_rate

    # report the alive cells
    self.report(self.alive, 'tth')
    self.alive = self.alive == 1
    self.defined = defined(self.alive)

    
    self.ort_session = ort.InferenceSession("cnn_game_of_life2000.onnx")
    self.input_name = self.ort_session.get_inputs()[0].name

  def dynamic(self):
    runtime_GameOfLife(self, 'tth')

nrOfTimeSteps = 100
myModel = GameOfLife()
dynamicModel = DynamicFramework(myModel, nrOfTimeSteps)
dynamicModel.run()

alive = readmap('tth.map')
aliveml_np = pcr2numpy(alive, 2)

plt.figure()
plt.imshow(aliveml_np)
plt.title(f'ML model 2000 - Time Step 0')
plt.show()

# visualize the results for each time step
for i in range(plot_time_step):
    # read the PCRaster map file for the current time step
    raster = readmap(f'tth00000.{i+1:03d}')

    # retrieve the raster data as a numpy array
    raster_data = pcr2numpy(raster, 2)

    # plot the raster data for the current time step
    plt.figure()
    plt.imshow(raster_data)
    plt.title(f'ML model 2000 - Time Step {i+1}')
    plt.show()

In [ ]:
y_true = np.zeros(shape=(100, 20, 20))
y_pred = np.zeros(shape=(100, 20, 20))

for i in range(1, 101): 
    # True labels
    true = readmap(f'testpcr0.{i:03d}')
    y_true[i-1] = pcr2numpy(true, 999)
    # Predicted labels
    pred = readmap(f'tth00000.{i:03d}')
    y_pred[i-1] = pcr2numpy(pred, 999)

y_true = y_true.reshape(-1)
y_pred = y_pred.reshape(-1)

# Calculate confusion matrix
cm = confusion_matrix(y_true, y_pred)
tn, fp, fn, tp = cm.ravel()

# Calculate metrics
accuracy = (tp + tn) / (tp + tn + fn + fp)
recall = tp / (tp + fn)
specificity = tn / (tn + fp)

# Print the results
print(f"Accuracy: {np.mean(accuracy):.2f}")
print(f"Recall: {np.mean(recall):.2f}")
print(f"Specificity: {np.mean(specificity):.2f}")

In [ ]:
accuracy_sum2000 = []
specificity_sum2000 = []
recall_sum2000 = []

for i in range(1, 101):
    print('time step:', i)
    truth = readmap(f'testpcr0.{i:03d}')
    truth_np = pcr2numpy(truth, 2)

    test = readmap(f'tth00000.{i:03d}')
    test_np = pcr2numpy(test, 2)

    # confusion matrix
    tn, fp, fn, tp = confusion_matrix(truth_np.flatten(), test_np.flatten()).ravel()

    # accuracy
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    accuracy_sum2000.append(accuracy)

    # recall
    recall = tp / (tp + fn)
    recall_sum2000.append(recall)

    # specificity
    specificity = tn / (tn + fp)
    specificity_sum2000.append(specificity)

    print(f"Accuracy: {round(accuracy, 2)}")
    print(f"Recall: {round(recall, 2)}")
    print(f"Specificity: {round(specificity, 2)}")
    print(' ')

print("Average accuracy:", round(np.mean(accuracy_sum2000), 2))
print("Average recall:", round(np.mean(recall_sum2000), 2))
print("Average Specificity:", round(np.mean(specificity_sum2000), 2))

In [ ]:
# Plotting the accuracy data
plt.figure(figsize=(10, 5))
plt.plot(accuracy_sum2000, marker='o', linestyle='-', color='b')
plt.title('Model Accuracy Over Time - Data Size 2000')
plt.xlabel('Time Step')
plt.ylabel('Accuracy')
plt.grid(True)

# Plotting the recall data
plt.figure(figsize=(10, 5))
plt.plot(recall_sum2000, marker='o', linestyle='-', color='r')
plt.title('Model Recall Over Time - Data Size 2000')
plt.xlabel('Time Step')
plt.ylabel('Recall')
plt.grid(True)

# Plotting the specificity data
plt.figure(figsize=(10, 5))
plt.plot(specificity_sum2000, marker='o', linestyle='-', color='g')
plt.title('Model Specificity Over Time - Data Size 2000')
plt.xlabel('Time step')
plt.ylabel('Specificity')
plt.grid(True)
plt.show()

### Data size 20000

In [ ]:
class GameOfLife(DynamicModel):
  def __init__(self):
    DynamicModel.__init__(self)
    setclone('clone.map')
  
  def initial(self):

    setclone(20, 20, 1.0, 0, 0)
    
    pcraster._pcraster.setrandomseed(seed_number) # set seed
    aUniformMap = uniform(1)
    self.report(aUniformMap,'aUniformMap')
    self.alive = aUniformMap < alive_rate

    # report the alive cells
    self.report(self.alive, 'twth')
    self.alive = self.alive == 1
    self.defined = defined(self.alive)    
    
    self.ort_session = ort.InferenceSession("cnn_game_of_life20000.onnx")
    self.input_name = self.ort_session.get_inputs()[0].name

  def dynamic(self):
    runtime_GameOfLife(self, 'twth')

nrOfTimeSteps = 100
myModel = GameOfLife()
dynamicModel = DynamicFramework(myModel, nrOfTimeSteps)
dynamicModel.run()

alive = readmap('twth.map')
aliveml_np = pcr2numpy(alive, 2)

plt.figure()
plt.imshow(aliveml_np)
plt.title(f'ML model 20000 - Time Step 0')
plt.show()

# visualize the results for each time step
for i in range(plot_time_step):
    # read the PCRaster map file for the current time step
    raster = readmap(f'twth0000.{i+1:03d}')

    # retrieve the raster data as a numpy array
    raster_data = pcr2numpy(raster, 2)

    # plot the raster data for the current time step
    plt.figure()
    plt.imshow(raster_data)
    plt.title(f'ML model 20000 - Time Step {i+1}')
    plt.show()

In [ ]:
y_true = np.zeros(shape=(100, 20, 20))
y_pred = np.zeros(shape=(100, 20, 20))

for i in range(1, 101): 
    # True labels
    true = readmap(f'testpcr0.{i:03d}')
    y_true[i-1] = pcr2numpy(true, 999)

    # Predicted labels
    pred = readmap(f'twth0000.{i:03d}')
    y_pred[i-1] = pcr2numpy(pred, 999)


y_true = y_true.reshape(-1)
y_pred = y_pred.reshape(-1)

# Calculate confusion matrix
cm = confusion_matrix(y_true, y_pred)
tn, fp, fn, tp = cm.ravel()

# Calculate metrics
accuracy = (tp + tn) / (tp + tn + fn + fp)
recall = tp / (tp + fn)
specificity = tn / (tn + fp)

# Print the results
print(f"Accuracy: {np.mean(accuracy):.2f}")
print(f"Recall: {np.mean(recall):.2f}")
print(f"Specificity: {np.mean(specificity):.2f}")

In [ ]:
accuracy_sum20000 = []
specificity_sum20000 = []
recall_sum20000 = []

for i in range(1, 101):
    print('time step:', i)
    truth = readmap(f'testpcr0.{i:03d}')
    truth_np = pcr2numpy(truth, 2)

    test = readmap(f'twth0000.{i:03d}')
    test_np = pcr2numpy(test, 2)

    # confusion matrix
    tn, fp, fn, tp = confusion_matrix(truth_np.flatten(), test_np.flatten()).ravel()

    # accuracy
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    accuracy_sum20000.append(accuracy)

    # recall
    recall = tp / (tp + fn)
    recall_sum20000.append(recall)

    # specificity
    specificity = tn / (tn + fp)
    specificity_sum20000.append(specificity)

    print(f"Accuracy: {round(accuracy, 2)}")
    print(f"Recall: {round(recall, 2)}")
    print(f"Specificity: {round(specificity, 2)}")
    print(' ')

print("Average accuracy:", round(np.mean(accuracy_sum20000), 2))
print("Average recall:", round(np.mean(recall_sum20000), 2))
print("Average Specificity:", round(np.mean(specificity_sum20000), 2))

In [ ]:
# Plotting the accuracy data
plt.figure(figsize=(10, 5))
plt.plot(accuracy_sum20000, marker='o', linestyle='-', color='b')
plt.title('Model Accuracy Over Time - Data Size 20000')
plt.xlabel('Time Step')
plt.ylabel('Accuracy')
plt.grid(True)

# Plotting the recall data
plt.figure(figsize=(10, 5))
plt.plot(recall_sum20000, marker='o', linestyle='-', color='r')
plt.title('Model Recall Over Time - Data Size 20000')
plt.xlabel('Time Step')
plt.ylabel('Recall')
plt.grid(True)

# Plotting the specificity data
plt.figure(figsize=(10, 5))
plt.plot(specificity_sum20000, marker='o', linestyle='-', color='g')
plt.title('Model Specificity Over Time - Data Size 20000')
plt.xlabel('Time step')
plt.ylabel('Specificity')
plt.grid(True)
plt.show()

## Data size 3500

In [ ]:
class GameOfLife(DynamicModel):
  def __init__(self):
    DynamicModel.__init__(self)
    setclone('clone.map')
  
  def initial(self):

    setclone(20, 20, 1.0, 0, 0)
    
    pcraster._pcraster.setrandomseed(seed_number) # set seed
    aUniformMap = uniform(1)
    self.report(aUniformMap,'aUniformMap')
    self.alive = aUniformMap < alive_rate

    # report the alive cells
    self.report(self.alive, 'fth')
    self.alive = self.alive == 1
    self.defined = defined(self.alive)    
    
    self.ort_session = ort.InferenceSession("cnn_game_of_life3500.onnx")
    self.input_name = self.ort_session.get_inputs()[0].name

  def dynamic(self):
    runtime_GameOfLife(self, 'fth')

nrOfTimeSteps = 100
myModel = GameOfLife()
dynamicModel = DynamicFramework(myModel, nrOfTimeSteps)
dynamicModel.run()

alive = readmap('fth.map')
aliveml_np = pcr2numpy(alive, 2)

plt.figure()
plt.imshow(aliveml_np)
plt.title(f'ML model - Time Step 0')
plt.show()

# visualize the results for each time step
for i in range(plot_time_step):
    # read the PCRaster map file for the current time step
    raster = readmap(f'fth00000.{i+1:03d}')

    # retrieve the raster data as a numpy array
    raster_data = pcr2numpy(raster, 2)

    # plot the raster data for the current time step
    plt.figure()
    plt.imshow(raster_data)
    plt.title(f'ML model - Time Step {i+1}')
    plt.show()

In [ ]:
y_true = np.zeros(shape=(100, 20, 20))
y_pred = np.zeros(shape=(100, 20, 20))

for i in range(1, 101): 
    # True labels
    true = readmap(f'testpcr0.{i:03d}')
    y_true[i-1] = pcr2numpy(true, 999)

    # Predicted labels
    pred = readmap(f'twth0000.{i:03d}')
    y_pred[i-1] = pcr2numpy(pred, 999)


y_true = y_true.reshape(-1)
y_pred = y_pred.reshape(-1)

# Calculate confusion matrix
cm = confusion_matrix(y_true, y_pred)
tn, fp, fn, tp = cm.ravel()

# Calculate metrics
accuracy = (tp + tn) / (tp + tn + fn + fp)
recall = tp / (tp + fn)
specificity = tn / (tn + fp)

# Print the results
print(f"Accuracy: {np.mean(accuracy):.2f}")
print(f"Recall: {np.mean(recall):.2f}")
print(f"Specificity: {np.mean(specificity):.2f}")

# Confusion matrix result

In [ ]:
# Plotting the accuracy data
plt.figure(figsize=(10, 5))
plt.plot(accuracy_sum200, marker='o', linestyle='-', color='r', label='Dataset 200')
plt.plot(accuracy_sum2000, marker='o', linestyle='-', color='g', label='Dataset 2000')
plt.plot(accuracy_sum20000, marker='o', linestyle='-', color='b', label='Dataset 20000')
plt.title('Model Accuracy Over Time')
plt.xlabel('Time Step')
plt.ylabel('Accuracy')
plt.grid(True)
plt.legend()

# Plotting the recall data
plt.figure(figsize=(10, 5))
plt.plot(recall_sum200, marker='o', linestyle='-', color='r', label='Dataset 200')
plt.plot(recall_sum2000, marker='o', linestyle='-', color='g', label='Dataset 2000')
plt.plot(recall_sum20000, marker='o', linestyle='-', color='b', label='Dataset 20000')
plt.title('Model Recall Over Time')
plt.xlabel('Time Step')
plt.ylabel('Recall')
plt.grid(True)
plt.legend()

# Plotting the specificity data
plt.figure(figsize=(10, 5))
plt.plot(specificity_sum200, marker='o', linestyle='-', color='r', label='Dataset 200')
plt.plot(specificity_sum2000, marker='o', linestyle='-', color='g', label='Dataset 2000')
plt.plot(specificity_sum20000, marker='o', linestyle='-', color='b', label='ML Model')
plt.title('Model Specificity Over Time')
plt.xlabel('Time step')
plt.ylabel('Specificity')
plt.grid(True)
plt.legend()